In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from contextlib import redirect_stdout

%matplotlib inline
plt.rcParams["figure.figsize"] = (20,3)

from pmdarima.arima import auto_arima, ADFTest
from sklearn.metrics import mean_squared_error

import sys
import os

sys.path.append('C:\\Users\\vidis\\OneDrive\\Desktop\\Spring23\\BachelorProject\\Bachelor-project\\Functions')
from Merge import merge

In [13]:
df = merge('h16')
df.head()

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001,76.0


In [69]:
class SARIMA:
    def __init__(self, house, cons_ts = 500, prod_ts=3000, carb_ts=50):
        self.house = house
        self.cons_ts = cons_ts
        self.prod_ts = prod_ts
        self.carb_ts = carb_ts
    
        self.df = merge(self.house)
    
    def model(self, start_time, end_time, var):
        n = len(pd.date_range(start_time, end_time,freq='H'))
        
        if var=='prod':
            train_size = self.prod_ts
            string = var+'_'+str(self.house)
        elif var == 'cons':
            train_size = self.cons_ts
            string = var+'_'+str(self.house)
        else:
            train_size = self.carb_ts
            string = 'CO2Emission'
        
        train_ind = pd.date_range(end=start_time, periods=train_size+1, freq="H")
        train = self.df[string].loc[train_ind[0]:train_ind[-2]]
        test = self.df[string].loc[start_time:end_time]
        
        arima_model = auto_arima(train, start_p=0, start_d=0, start_q=0, 
                        max_p = 5, max_d=5, max_q=5, start_P=0,
                        start_D=0, start_Q=0, m=24, seasonal=True, 
                        error_action='warn', trace=True, supress_warning=True,
                        stepwise=True, random_state=20, n_fits=50)
        
        pred = arima_model.predict(n_periods=n)    
        rmse = np.sqrt(mean_squared_error(test,pred))
        
        tf = pd.DataFrame(columns=['test','pred'])
        tf['test'] = test
        tf['pred'] = pred
        return tf
    
    def SARIMA(self, start_time, end_time):
        pf = self.model(start_time, end_time, 'prod')
        cf = self.model(start_time, end_time, 'cons')
        carbf = self.model(start_time, end_time, 'carbon')
        price = self.df['SpotPriceDKK'].loc[start_time:end_time]
        
        of = pd.DataFrame()
        of['prod_'+str(self.house)] = pf['pred'].to_numpy()
        of['cons_'+str(self.house)] = cf['pred'].to_numpy()
        of['yield'] = of['prod_'+str(self.house)] - of['cons_'+str(self.house)]
        of['SpotPriceDKK'] = (self.df['SpotPriceDKK'].loc[start_time:end_time]).to_numpy()
        of['CO2Emission'] = carbf['pred'].to_numpy()
        
        of.set_index(df.loc[start_time:end_time].index,inplace=True)
        return of

In [70]:
h16 = SARIMA('h16')

In [71]:
h16.SARIMA('2022-06-20 04:00:00','2022-06-21 01:00:00')

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=15503.832, Time=0.08 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=14557.220, Time=1.88 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=14828.774, Time=2.01 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=15501.832, Time=0.11 sec
 ARIMA(1,1,0)(0,0,0)[24] intercept   : AIC=14993.016, Time=0.34 sec
 ARIMA(1,1,0)(2,0,0)[24] intercept   : AIC=14236.271, Time=11.64 sec
 ARIMA(1,1,0)(2,0,1)[24] intercept   : AIC=inf, Time=34.74 sec
 ARIMA(1,1,0)(1,0,1)[24] intercept   : AIC=inf, Time=7.34 sec
 ARIMA(0,1,0)(2,0,0)[24] intercept   : AIC=14234.282, Time=10.22 sec
 ARIMA(0,1,0)(1,0,0)[24] intercept   : AIC=14579.974, Time=1.67 sec
 ARIMA(0,1,0)(2,0,1)[24] intercept   : AIC=inf, Time=35.42 sec
 ARIMA(0,1,0)(1,0,1)[24] intercept   : AIC=inf, Time=4.76 sec
 ARIMA(0,1,1)(2,0,0)[24] intercept   : AIC=14236.273, Time=13.56 sec
 ARIMA(1,1,1)(2,0,0)[24] intercept   : AIC=inf, Time=73.95 sec
 ARIMA(0,1,0)(2,0,0)[24]     

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
2022-06-20 04:00:00,-3.654458e-19,4.419715,-4.419715,1585.989990,155.068112
2022-06-20 05:00:00,-8.819402e-19,4.454186,-4.454186,1958.459961,154.279334
2022-06-20 06:00:00,1.676944e-01,4.680814,-4.513120,2244.120117,153.611688
2022-06-20 07:00:00,5.021844e-01,4.680447,-4.178262,2753.469971,153.046571
2022-06-20 08:00:00,1.106244e+00,5.051305,-3.945061,3719.500000,152.568239
2022-06-20 09:00:00,2.830032e+00,6.234017,-3.403985,2590.850098,152.163364
2022-06-20 10:00:00,6.317357e+00,8.243643,-1.926286,2308.169922,151.820666
2022-06-20 11:00:00,7.058547e+00,6.731219,0.327328,2276.110107,151.530595
2022-06-20 12:00:00,8.053926e+00,7.221078,0.832848,1960.469971,151.285070
2022-06-20 13:00:00,9.702006e+00,6.722088,2.979918,1590.979980,151.077250
